- asdfg
    - sdfghj

In [15]:
import pandas as pd

In [3]:
pwd

'/Users/hchae/Desktop'

In [4]:
dataset = pd.read_csv("london_merged.csv")

In [5]:
dataset.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [6]:
!pip install pgmpy

     |████████████████████████████████| 1.9 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 147.1 MB 13.0 MB/s eta 0:00:01     |███████████████████████████     | 124.6 MB 27.8 MB/s eta 0:00:01


In [28]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator,BayesianEstimator
model = BayesianModel([("t1","t2")])

/Users/hchae/opt/anaconda3/lib/python3.8/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.
  warnings.warn(


In [30]:
model.fit(dataset, estimator = MaximumLikelihoodEstimator)

In [34]:
# calculate individual prob
print(model.get_cpds('t2'))

+------------------------+----------+-----+----------+----------+----------+
| t1                     | t1(-1.5) | ... | t1(33.0) | t1(33.5) | t1(34.0) |
+------------------------+----------+-----+----------+----------+----------+
| t2(-6.0)               | 0.0      | ... | 0.0      | 0.0      | 0.0      |
+------------------------+----------+-----+----------+----------+----------+
| t2(-5.5)               | 0.0      | ... | 0.0      | 0.0      | 0.0      |
+------------------------+----------+-----+----------+----------+----------+
| t2(-5.0)               | 0.0      | ... | 0.0      | 0.0      | 0.0      |
+------------------------+----------+-----+----------+----------+----------+
| t2(-4.5)               | 0.0      | ... | 0.0      | 0.0      | 0.0      |
+------------------------+----------+-----+----------+----------+----------+
| t2(-4.0)               | 0.0      | ... | 0.0      | 0.0      | 0.0      |
+------------------------+----------+-----+----------+----------+----------+

In [ ]:
print